<a href="https://colab.research.google.com/github/deepan-DA21/sample/blob/main/logistics-gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import gradio as gr
import numpy as np
import random

# Function to generate a random distance matrix
def generate_distance_matrix(n_cities):
    matrix = np.random.randint(10, 100, size=(n_cities, n_cities))
    np.fill_diagonal(matrix, 0)  # No distance to self
    return matrix

# Simple genetic algorithm for the TSP problem
def create_individual(n_cities):
    return random.sample(range(n_cities), n_cities)

def fitness(route, distance_matrix):
    return sum(distance_matrix[route[i], route[i+1]] for i in range(len(route) - 1)) + distance_matrix[route[-1], route[0]]  # Return to the start

def mutate(route):
    i, j = random.sample(range(len(route)), 2)
    route[i], route[j] = route[j], route[i]
    return route

def crossover(parent1, parent2):
    start, end = sorted(random.sample(range(len(parent1)), 2))
    child = [-1] * len(parent1)
    child[start:end] = parent1[start:end]

    current = end
    for i in range(len(parent2)):
        if parent2[i] not in child:
            if current == len(parent1):
                current = 0
            child[current] = parent2[i]
            current += 1
    return child

def genetic_algorithm(pop_size, generations, mutation_rate, distance_matrix):
    n_cities = len(distance_matrix)
    population = [create_individual(n_cities) for _ in range(pop_size)]

    for generation in range(generations):
        population.sort(key=lambda route: fitness(route, distance_matrix))
        new_population = population[:2]  # Keep the best two individuals

        while len(new_population) < pop_size:
            parent1, parent2 = random.sample(population[:10], 2)  # Select top 10 individuals
            child = crossover(parent1, parent2)
            if random.random() < mutation_rate:
                child = mutate(child)
            new_population.append(child)

        population = new_population

    best_route = population[0]
    best_distance = fitness(best_route, distance_matrix)
    return best_route, best_distance

# Function to be used in the Gradio interface
def tsp_genetic_algorithm(n_cities, pop_size, generations, mutation_rate):
    distance_matrix = generate_distance_matrix(n_cities)

    best_route, best_distance = genetic_algorithm(pop_size, generations, mutation_rate, distance_matrix)

    # Convert best route to a readable format
    route_str = " -> ".join(str(city) for city in best_route) + f" -> {best_route[0]}"  # Return to the start city
    return route_str, best_distance

# Gradio Interface
iface = gr.Interface(
    fn=tsp_genetic_algorithm,
    inputs=[
        gr.Slider(minimum=3, maximum=15, step=1, value=5, label="Number of Cities"),  # Change 'default' to 'value'
        gr.Slider(minimum=10, maximum=100, step=1, value=20, label="Population Size"),  # Change 'default' to 'value'
        gr.Slider(minimum=10, maximum=500, step=1, value=100, label="Generations"),  # Change 'default' to 'value'
        gr.Slider(minimum=0.01, maximum=1, step=0.01, value=0.1, label="Mutation Rate")  # Change 'default' to 'value'
    ],
    outputs=[
        gr.Textbox(label="Best Route"),
        gr.Textbox(label="Best Distance (Cost)")
    ],
    live=True,
    title="Genetic Algorithm for Traveling Salesman Problem (TSP)",
    description="Use the sliders to select the number of cities, population size, generations, and mutation rate. The genetic algorithm will compute the best route and its total distance."
)

iface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://af19262ea9493b2708.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
